In [10]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3


#生成者ネットワーク
generator_input = keras.Input(shape=(latent_dim,))


x = layers.Dense(128*16*16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16,16,128))(x)

x = layers.Conv2D(256,5,padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2,padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5,padding="same")(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5,padding="same")(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels,7,activation="tanh",padding="same")(x)

generator = keras.models.Model(generator_input,x)
generator.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [11]:
#判別者ネットワーク
discriminator_input = layers.Input(shape=(height,width,channels))

x = layers.Conv2D(128,3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x) 
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x) 
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x) 
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1,activation="sigmoid")(x)

discriminator = keras.models.Model(discriminator_input,x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(lr=0.008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss="binary_crossentropy")



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [12]:
#敵対者ネットワーク　生成者と判別者をつなぎ合わせる

discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))

gan = keras.models.Model(gan_input,gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.004,clipvalue=1.0,decay=1e-8)

gan.compile(optimizer=gan_optimizer,loss="binary_crossentropy")



In [ ]:
import os 
from keras.preprocessing import image

(x_train,y_train),(_,_) = keras.datasets.cifar10.load_data()

#choose the images of frogs
x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape((x_train.shape[0],)+(height,width,channels)).astype("float32") / 255.

iterations = 10000
batch_size = 20

save_dir = "C:/Users/kenta takizawa/Desktop/test"


In [ ]:


start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = x_train[start:stop]
    combined_images = np.concatenate([generated_images,real_images])
    
    labels = np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images,labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    
    misleading_targets = np.zeros((batch_size,1))
    
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start += batch_size
    
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        #gan.save_weights('gan.h5')
        
        print("discriminator loss at step %s: %s" % (step,d_loss))
        print("adversarial loss at step %s: %s" % (step,a_loss))
        
        img2 = image.array_to_img(generated_images[0] * 255. , scale=False)
        img2.save(os.path.join(save_dir,"generated_frog"+str(step)+".png"))
        
        img3 = image.array_to_img(real_images[0]*255.,scale=False)
        img3.save(os.path.join(save_dir,"real_flog"+str(step)+".png"))
                 
                 

discriminator loss at step 0: 8.495903
adversarial loss at step 0: 1.0000001e-07
discriminator loss at step 100: 8.376001
adversarial loss at step 100: 1.0000001e-07
discriminator loss at step 200: 8.437133
adversarial loss at step 200: 1.0000001e-07
discriminator loss at step 300: 8.47573
adversarial loss at step 300: 1.0000001e-07
discriminator loss at step 400: 8.474376
adversarial loss at step 400: 1.0000001e-07
discriminator loss at step 500: 8.401718
adversarial loss at step 500: 1.0000001e-07
discriminator loss at step 600: 8.48398
adversarial loss at step 600: 1.0000001e-07
discriminator loss at step 700: 8.449086
adversarial loss at step 700: 1.0000001e-07
discriminator loss at step 800: 8.490149
adversarial loss at step 800: 1.0000001e-07
discriminator loss at step 900: 8.427663
adversarial loss at step 900: 1.0000001e-07
discriminator loss at step 1000: 8.438995
adversarial loss at step 1000: 1.0000001e-07
discriminator loss at step 1100: 8.479214
adversarial loss at step 11

In [16]:
x_train.shape

(5000, 32, 32, 3)

In [9]:
real_images.shape

(20, 32, 32, 3)